## Part 1

### Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Scrapping data

We are retreiving the URL and creating a Beautiful soup object create three lists.

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

After finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.

Append the data to the list.

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


Make a dataframe from the above lists.

In [5]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
print(df)

    PostalCode                 Borough  \
0          M3A              North York   
1          M4A              North York   
2          M5A        Downtown Toronto   
3          M6A              North York   
4          M7A            Queen's Park   
5          M9A               Etobicoke   
6          M1B             Scarborough   
7          M3B              North York   
8          M4B               East York   
9          M5B        Downtown Toronto   
10         M6B              North York   
11         M9B               Etobicoke   
12         M1C             Scarborough   
13         M3C              North York   
14         M4C               East York   
15         M5C        Downtown Toronto   
16         M6C                    York   
17         M9C               Etobicoke   
18         M1E             Scarborough   
19         M4E            East Toronto   
20         M5E        Downtown Toronto   
21         M6E                    York   
22         M1G             Scarbor

In [7]:
df.shape

(103, 3)

## Part 2


In [8]:
df1 = pd.read_csv("C:/Users/Desktop/DSML/DSPC/10. Capstone/Geospatial_Coordinates.csv")

In [9]:
df["Latitude"] = df1["Latitude"]
df["Longitude"] = df1["Longitude"]

In [10]:
print(df)

    PostalCode                 Borough  \
0          M3A              North York   
1          M4A              North York   
2          M5A        Downtown Toronto   
3          M6A              North York   
4          M7A            Queen's Park   
5          M9A               Etobicoke   
6          M1B             Scarborough   
7          M3B              North York   
8          M4B               East York   
9          M5B        Downtown Toronto   
10         M6B              North York   
11         M9B               Etobicoke   
12         M1C             Scarborough   
13         M3C              North York   
14         M4C               East York   
15         M5C        Downtown Toronto   
16         M6C                    York   
17         M9C               Etobicoke   
18         M1E             Scarborough   
19         M4E            East Toronto   
20         M5E        Downtown Toronto   
21         M6E                    York   
22         M1G             Scarbor

In [11]:
df.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East Toronto Business,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054


### Filtering the dataframe for "Toronto"

In [12]:
Toronto_df = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(Toronto_df.shape)

(39, 5)


In [13]:
print(Toronto_df)

   PostalCode                 Borough  \
0         M5A        Downtown Toronto   
1         M5B        Downtown Toronto   
2         M5C        Downtown Toronto   
3         M4E            East Toronto   
4         M5E        Downtown Toronto   
5         M5G        Downtown Toronto   
6         M6G        Downtown Toronto   
7         M5H        Downtown Toronto   
8         M6H            West Toronto   
9         M4J  East York/East Toronto   
10        M5J        Downtown Toronto   
11        M6J            West Toronto   
12        M4K            East Toronto   
13        M5K        Downtown Toronto   
14        M6K            West Toronto   
15        M4L            East Toronto   
16        M5L        Downtown Toronto   
17        M4M            East Toronto   
18        M4N         Central Toronto   
19        M5N         Central Toronto   
20        M4P         Central Toronto   
21        M5P         Central Toronto   
22        M6P            West Toronto   
23        M4R   

In [14]:
Toronto_df.drop_duplicates(subset ="PostalCode",keep = "first", inplace = True)
Toronto_df.reset_index(drop=True) 
print(Toronto_df.shape)

(39, 5)


## Part 3

### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Creating map of neighborhoods in Toronto

Let's get the geographical coordinates of Manhattan.

In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let's create the map with the neighbourhoods.

In [17]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'],Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Showcasing map if not displayed in Github.

In [50]:

from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/josephbenofficial/coursera-capstone/blob/master/Week%203/torontobeforecluster.PNG?raw=true")

### Exploring the first neighbourhood & get the top 100 venues that are within a radius of 500 meters of the first neighbourhood

In [18]:
Toronto_df.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [19]:
neighborhood_latitude = Toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [20]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c79de1948d0065fba84d70'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        '

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

 Clean the json and structure it into a pandas dataframe.

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-23-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Explore all neighborhoods in Toronto

We create a function to repeat the same process to all the neighborhoods in Toronto.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called Torontovenues.

In [26]:
Torontovenues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [27]:
print(Torontovenues.shape)

(713, 7)


In [28]:
Torontovenues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
5,"Regent Park, Harbourfront",43.763573,-79.188711,Krispy Kreme Doughnuts,43.767169,-79.189660,Donut Shop
6,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Regent Park, Harbourfront",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
8,"Regent Park, Harbourfront",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,"Garden District, Ryerson",43.692657,-79.264848,Birchmount Community Centre,43.695175,-79.262161,General Entertainment


In [29]:
Torontovenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",28,28,28,28,28,28
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,6,6,6,6,6,6
Christie,3,3,3,3,3,3
Church and Wellesley,9,9,9,9,9,9
Davisville,5,5,5,5,5,5
Davisville North,61,61,61,61,61,61
"Dufferin, Dovercourt Village",5,5,5,5,5,5
Enclave of M4L,4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues.

In [30]:
print('There are {} uniques categories.'.format(len(Torontovenues['Venue Category'].unique())))

There are 184 uniques categories.


### Analyze Each Neighborhood

In [31]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Torontovenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Torontovenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
Toronto_onehot.shape

(713, 184)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.035714,0.035714,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.0000,0.00,0.00000,0.035714,0.000000,0.000000,0.000000,0.000000,0.107143,0.00,0.0,0.035714,0.000000,0.0,0.035714,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.035714,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.035714,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.035714,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.00000,0.00,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.000000,0.00,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000

In [34]:
Toronto_grouped.shape

(37, 184)

### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.07
2    Gastropub  0.07
3       Bakery  0.07
4         Park  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Smoke Shop  0.06
3       Garden Center  0.06
4              Garden  0.06


----Central Bay Street----
            venue  freq
0   Grocery Store  0.17
1     Coffee Shop  0.17
2        Pharmacy  0.17
3     Pizza Place  0.17
4  Discount Store  0.17


----Christie----
                       venue  freq
0                       Park  0.33
1       Fast Food Restaurant  0.33
2          Food & Drink Shop  0.33
3  Latin American Restaurant  0.00
4             Lingerie Store  0.00


----Church and Wellesley----
                venue  freq
0         Pizza Place  0.22
1         Coffee Shop  0.11
2  Chinese Restaura

Let's put that into a pandas dataframe.
1. First, let's write a function to sort the venues in descending order.
2. Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [36]:
#1
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
#2
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Gastropub,Bakery,Park,Furniture / Home Store,Middle Eastern Restaurant,Electronics Store,Bookstore,Fish Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Smoke Shop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place
2,Central Bay Street,Grocery Store,Coffee Shop,Pharmacy,Pizza Place,Discount Store,Supermarket,Yoga Studio,Miscellaneous Shop,Lounge,Martial Arts School
3,Christie,Park,Fast Food Restaurant,Food & Drink Shop,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
4,Church and Wellesley,Pizza Place,Coffee Shop,Chinese Restaurant,Playground,Discount Store,Sandwich Place,Middle Eastern Restaurant,Intersection,Mobile Phone Shop,Martial Arts School


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters. 

In [38]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 1, 3, 3, 3, 3, 1, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,3.0,Restaurant,Donut Shop,Medical Center,Breakfast Spot,Bank,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Middle Eastern Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,3.0,General Entertainment,Café,Farm,Skating Rink,College Stadium,New American Restaurant,Nail Salon,Lingerie Store,Liquor Store,Lounge
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,3.0,Fast Food Restaurant,Discount Store,Bank,Electronics Store,Intersection,Camera Store,Indian Restaurant,Sandwich Place,Pizza Place,Chinese Restaurant
3,M4E,East Toronto,The Beaches,43.786947,-79.385975,3.0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Mobile Phone Shop,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's visualize the resulting clusters.

In [40]:
print(Toronto_merged)


   PostalCode                 Borough  \
0         M5A        Downtown Toronto   
1         M5B        Downtown Toronto   
2         M5C        Downtown Toronto   
3         M4E            East Toronto   
4         M5E        Downtown Toronto   
5         M5G        Downtown Toronto   
6         M6G        Downtown Toronto   
7         M5H        Downtown Toronto   
8         M6H            West Toronto   
9         M4J  East York/East Toronto   
10        M5J        Downtown Toronto   
11        M6J            West Toronto   
12        M4K            East Toronto   
13        M5K        Downtown Toronto   
14        M6K            West Toronto   
15        M4L            East Toronto   
16        M5L        Downtown Toronto   
17        M4M            East Toronto   
18        M4N         Central Toronto   
19        M5N         Central Toronto   
20        M4P         Central Toronto   
21        M5P         Central Toronto   
22        M6P            West Toronto   
23        M4R   

In [41]:
Toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)

In [42]:
print(Toronto_merged)


   PostalCode                 Borough  \
0         M5A        Downtown Toronto   
1         M5B        Downtown Toronto   
2         M5C        Downtown Toronto   
3         M4E            East Toronto   
5         M5G        Downtown Toronto   
6         M6G        Downtown Toronto   
7         M5H        Downtown Toronto   
8         M6H            West Toronto   
9         M4J  East York/East Toronto   
10        M5J        Downtown Toronto   
11        M6J            West Toronto   
12        M4K            East Toronto   
13        M5K        Downtown Toronto   
14        M6K            West Toronto   
15        M4L            East Toronto   
17        M4M            East Toronto   
18        M4N         Central Toronto   
19        M5N         Central Toronto   
20        M4P         Central Toronto   
21        M5P         Central Toronto   
22        M6P            West Toronto   
23        M4R         Central Toronto   
24        M5R         Central Toronto   
25        M6R   

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'],Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Had to eliminate null values in Toronto_merged and use int(cluster). Was getting error: list indices must be integers or slices, not float.

Showcasing map if not displayed in Github.

In [49]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/josephbenofficial/coursera-capstone/blob/master/Week%203/torontoaftercluster.PNG?raw=true")

### Examine Clusters

Cluster 0

In [44]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,0.0,Baseball Field,Yoga Studio,Miscellaneous Shop,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant
36,Downtown Toronto,0.0,Baseball Field,Yoga Studio,Miscellaneous Shop,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant


Cluster 1


In [45]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
7,Downtown Toronto,1.0,Park,Business Service,Airport,Wine Bar,Office,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School
24,Central Toronto,1.0,Park,Pool,Women's Store,Wine Bar,Vietnamese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School
38,East Toronto Business,1.0,Park,Mobile Phone Shop,Sandwich Place,Bus Line,New American Restaurant,Middle Eastern Restaurant,Liquor Store,Lounge,Martial Arts School,Medical Center


Cluster 2

In [46]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,West Toronto,2.0,Auto Workshop,Convenience Store,Brewery,Yoga Studio,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant


Cluster 3

In [47]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3.0,Restaurant,Donut Shop,Medical Center,Breakfast Spot,Bank,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Middle Eastern Restaurant
1,Downtown Toronto,3.0,General Entertainment,Café,Farm,Skating Rink,College Stadium,New American Restaurant,Nail Salon,Lingerie Store,Liquor Store,Lounge
2,Downtown Toronto,3.0,Fast Food Restaurant,Discount Store,Bank,Electronics Store,Intersection,Camera Store,Indian Restaurant,Sandwich Place,Pizza Place,Chinese Restaurant
3,East Toronto,3.0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Mobile Phone Shop,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
5,Downtown Toronto,3.0,Grocery Store,Coffee Shop,Pharmacy,Pizza Place,Discount Store,Supermarket,Yoga Studio,Miscellaneous Shop,Lounge,Martial Arts School
8,West Toronto,3.0,Park,Hotel,Grocery Store,Shopping Mall,Bank,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
9,East York/East Toronto,3.0,Pizza Place,Pharmacy,Athletics & Sports,Intersection,Flea Market,Bank,Café,Gastropub,Gym / Fitness Center,Nail Salon
10,Downtown Toronto,3.0,Skating Rink,Park,Beer Store,Spa,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center
11,West Toronto,3.0,Pizza Place,Health Food Store,Pub,Yoga Studio,Mobile Phone Shop,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant
12,East Toronto,3.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Juice Bar,Frozen Yogurt Shop,Japanese Restaurant,Spa,Pub


Cluster 4

In [48]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Downtown Toronto,4.0,Clothing Store,Food Truck,Office,Mobile Phone Shop,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Mexican Restaurant


### The End